In [ ]:
# -*- coding: utf-8 -*-
"""
Script to invert Tongariro SP data using divergence of streaming current density, q.

from Soegi Kang, Dominique Fournier, UBC, Craig Miller, GNS

https://github.com/simpeg

please use the miller_tongariro_2018 branch

"""
from SimPEG import Utils, Maps, Mesh
from SimPEG.EM.Static import SP
import matplotlib.pyplot as plt
import numpy as np
from SimPEG.EM.Static.Utils import StaticUtils
import os
from sklearn.metrics import mean_squared_error as mse
from matplotlib.ticker import AutoMinorLocator
import colorcet as cc
from SimPEG.Utils import download
import tarfile

tar_folder = download( "https://storage.googleapis.com/simpeg/miller_et_al_2018.tar.gz")  
tar = tarfile.open(tar_folder, "r:*")
tar.extractall()
tar.close()

datadir ="./miller_et_al_2018/"

plt.close('all')


In [ ]:
# load files

topo = np.loadtxt(datadir+'topoxyz.dat', skiprows=1)
data = np.loadtxt(datadir+'Tongariro_SP_data.dat')

cond_model_file = datadir+'MT_model_for_SP_inv.con'
mesh_file = datadir+'Tongariro_SP.msh'


In [ ]:
dataInv=data
topo = topo[(topo[:,0] < 1833000) & (topo[:,0] > 1824000)]
topo = topo[(topo[:,1] > 5662000) & (topo[:,1] < 5670000)]
topoInv = topo


In [ ]:
#set up the mesh
mesh = Mesh.TensorMesh.readUBC(mesh_file)

print( mesh)
print( 'width in X direction', mesh.hx.sum(), 'm')
print( 'width in Y direction', mesh.hy.sum(), 'm')
print( 'width in Z direction', mesh.hz.sum(), 'm')
print( 'number of cells', mesh.nC)

xref, yref = 1830000, 5666000


In [ ]:
#sigma is background conductivity (S/m)

#read in model file
resistivity = mesh.readModelUBC(cond_model_file) #read background conductivity model from file
sigma = 1/resistivity #change to S/m from ohmm
actind = sigma != 1e-8
#sigma = np.ones(mesh.nC)*1e-3 # sets uniform background
sigma[~actind] = 1e-8 #sigma of cells above topo = 1e-8


In [ ]:
#set active cells
actMap = Maps.InjectActiveCells(mesh, actind, -100.) #makes cell below topo active and sets cells above topo to 0
mesh2D, topoCC = StaticUtils.gettopoCC(mesh, actind) #~actind = opposite of actind, ie swaps True for False


In [ ]:
#drape points onto topo
xyzlocInv = StaticUtils.drapeTopotoLoc(mesh, dataInv[:,:2], actind=actind) #drape data onto topo.
xyzlocInv = np.c_[xyzlocInv[:,:2], xyzlocInv[:,2]]


In [ ]:
#  Inversion runs in the next sections.
# Setup the problem

from pymatsolver import PardisoSolver

ind = np.argmin(abs(xyzlocInv[:, 0]-xref)+abs(xyzlocInv[:, 1]-yref))
inds = np.ones(xyzlocInv.shape[0], dtype='bool')
inds[ind] = False
xyzM = xyzlocInv[inds, :]
xyzN = np.atleast_2d(xyzlocInv[ind]).repeat(xyzM.shape[0], axis=0)
#wires = Maps.Wires(('jsx', actMap.nP), ('jsy', actMap.nP), ('jsz', actMap.nP))
prob = SP.Problem_CC_Jstore(mesh, sigma=sigma, qMap=actMap, Solver=PardisoSolver)
rx = SP.Rx.Dipole(xyzM, xyzN)
src = SP.Src.StreamingCurrents([rx], L=np.ones(mesh.nC), mesh=mesh, modelType="CurrentSource")
survey = SP.SurveySP_store([src])
survey.pair(prob)

survey.dobs = (dataInv[1:,2]-dataInv[0,2])  # mV


plt.hist(survey.dobs, bins=100)
plt.show()


In [ ]:
from SimPEG import (Mesh, Maps, DataMisfit, Regularization,
                    Optimization, Inversion, InvProblem, Directives)

survey.std = 30 #survey std dev in mV
survey.eps = 60 # survey noise floor in mV


dmisfit = DataMisfit.l2_DataMisfit(survey)
regmap = Maps.IdentityMap(nP = actMap.nP)
reg = Regularization.Sparse(mesh, mapping=regmap, indActive=actind)

reg.alpha_s = 1

reg.norms = np.c_[0, 1, 1, 1]

opt = Optimization.ProjectedGNCG(maxIter=100, maxIterCG=10, tolX=1e-20, tolF=1e-20)

opt.maxIterLS = 20

invProb = InvProblem.BaseInvProblem(dmisfit, reg, opt)
target = Directives.TargetMisfit()

# Create an inversion object
update_Jacobi = Directives.UpdatePreconditioner()
update_SensWeight = Directives.UpdateSensitivityWeights()

IRLS = Directives.Update_IRLS(
        f_min_change=1e-3, minGNiter=2, maxIRLSiter = 10,
        coolingFactor=8, coolingRate=3
        )


betaest = Directives.BetaEstimate_ByEig(beta0_ratio=1.)
updateprecond = Directives.UpdatePreconditioner()

#LP only
inv = Inversion.BaseInversion(invProb, directiveList=[betaest, IRLS, updateprecond, update_SensWeight])

#save iterations
prob.counter = opt.counter = Utils.Counter()
opt.LSshorten = 0.5
opt.remember('xc')

#creates an initial starting model m0
m0 = np.ones(actMap.nP)*0.
reg.mref = m0



In [ ]:
#run the inversion
mopt_Lp = inv.run(m0)
Lp_amp = actMap*mopt_Lp

#recall a particular iteration, comment out if not needed.
xc = opt.recall('xc')
mopt_Lp = xc[8]
Lp_amp = actMap*mopt_Lp


In [ ]:
#save models and calc data

mesh.writeModelUBC('Tongairo_C_Lp_amp.mod', Lp_amp)

#Write calculated data (Lp model)
Lp_calculated = survey.dpred(mopt_Lp)
np.savetxt('Lp_calculated_data.dat', Lp_calculated, fmt='%.4f')


RMS = np.sqrt(mse(Lp_calculated, survey.dobs))
print('RMS = %.3f mV' %RMS)


In [ ]:
# Plot some L2 model obs and calc
plt.figure()
plt.plot(survey.dobs, '.', label = 'obs')
plt.plot(invProb.dpred, label = 'calc')
plt.legend()
plt.ylabel('mV')
plt.xlabel('data point index')
plt.annotate('B', xy=(0, 1000), fontsize=14)
plt.title('$q$ observed vs calculated.  RMS = %.0f mV' %RMS)
plt.show()

plt.figure()
plt.hist(mopt_Lp, bins=100, label='Lp model')
plt.legend()
plt.show()




In [ ]:
# extract core mesh
temp = Lp_amp.copy()
temp[~actind] = np.nan
vmin, vmax = temp[actind].min(), temp[actind].max()

xmin, xmax = dataInv[:,0].min(), dataInv[:,0].max()
ymin, ymax = dataInv[:,1].min(), dataInv[:,1].max()
zmin, zmax = 700, topo[:,2].max()
xyzlim = np.array([[xmin, xmax], [ymin, ymax], [zmin, zmax]])
ind_core, mesh_core = Utils.ExtractCoreMesh(xyzlim, mesh)

ja_est_core = Utils.mkvc(temp[ind_core])
actind_core = actind[ind_core]
jamp_min, jamp_max = ja_est_core[:mesh_core.nC][actind_core].min(), ja_est_core[:mesh_core.nC][actind_core].max()
jamp_lp_core = temp[ind_core]
jamp_lp_core[~actind_core] = np.nan


In [ ]:
# plot map views
fig, ((ax0,ax1),(ax2,ax3)) = plt.subplots(nrows=2, ncols=2, figsize = (10, 10))

fig.subplots_adjust(wspace=0, hspace=0)
axs = [ax0, ax1, ax2, ax3]
layers = [1500, 1200, 1000, 800]


clim=(-0.00005, 0.00005)
for i_layer, ax in enumerate(axs):   
    jamp_lp_core[~actind_core] = np.nan
    indz = int(np.argmin(abs(mesh_core.vectorCCz-layers[i_layer])))
    cbar = mesh_core.plotSlice(jamp_lp_core, ind=indz, normal='Z', clim=clim, ax=ax)
    
    minorLocator = AutoMinorLocator(4)
    ax.xaxis.set_minor_locator(minorLocator)
    ax.yaxis.set_minor_locator(minorLocator)

    ax.tick_params(which='both', width=2, direction='in')
    ax.tick_params(which='major', length=10, direction='in', top= True, right=True)
    ax.tick_params(which='minor', length=7, direction='in', top = True)
    
    ax.set_title('')
    ax.set_xlabel('')
    ax.set_ylabel('')
    ax.set_aspect('equal')
    ax.set_ylim(5663000, 5669000)
    ax.set_xlim(1825094, 1831794)    
    ax.set_title(("%.0f")%(layers[i_layer]))
    ax.set_xticks(np.arange(1826000, 1832000, 2000))
    ax.set_yticks(np.arange(5663000, 5670000, 2000))
    
    if ~np.logical_or(i_layer==2,i_layer==3):
        ax.set_xticklabels([])
    if np.logical_or(i_layer==1,i_layer==3):
        ax.set_yticklabels({})

plt.tight_layout()
plt.show()


In [ ]:
# plot section views
fig, ((ax0,ax1),(ax2,ax3),(ax4,ax5)) = plt.subplots(nrows=3, ncols=2, figsize = (10, 10))

fig.delaxes(ax5)

fig.subplots_adjust(wspace=0, hspace=0)
axs = [ax0, ax1, ax2, ax3, ax4, ax5]
layers = [5667994, 5666994, 5665994, 5664994, 5663994]

stream_color = 'k'
clim = (-0.00005, 0.00005)
for i_layer, ax in enumerate(axs[:-1]):
    jamp_lp_core[~actind_core] = np.nan
    indy = int(np.argmin(abs(mesh_core.vectorCCy-layers[i_layer])))
    cbar = mesh_core.plotSlice(jamp_lp_core, ind=indy, normal='Y', clim=clim, ax=ax)

    minorLocator = AutoMinorLocator(4)
    ax.xaxis.set_minor_locator(minorLocator)
    ax.yaxis.set_minor_locator(minorLocator)

    ax.tick_params(which='both', width=2, direction='in')
    ax.tick_params(which='major', length=10, direction='in', top= True, right=True)
    ax.tick_params(which='minor', length=7, direction='in', top = True)
    
    ax.set_title('')
    ax.set_xlabel('')
    ax.set_ylabel('')
    ax.set_aspect('equal')
    ax.set_ylim(700, 2000)
    ax.set_xlim(1825094, 1831794)    
    ax.set_title(("%.0f")%(layers[i_layer]))
    ax.set_xticks([1826000, 1828000, 1830000])
    ax.set_yticks([1000, 1500, 2000])
      
    if ~np.logical_or(i_layer==4,i_layer==3) :
        ax.set_xticklabels([])
    if np.logical_or(i_layer==1,i_layer==3):
        ax.set_yticklabels({})

plt.tight_layout()
plt.show()



In [ ]:
# Write core of the L2 model to file in spherical formulation
mesh_core.writeUBC('mesh_core.msh')

jamp_lp_core[~actind_core] = -100
mesh_core.writeModelUBC('Tongairo_C_Lp_amp_core.mod', jamp_lp_core)